## [웹크롤링 _ 나무위키 사이트 분석 및 시각화]
### <Step1. 크롤링> : 크롤링으로 웹 데이터 가져오기

[웹크롤링 라이브러리 사용하기]
- 파이썬에서는 BeautifulSoup와 requests 라는 라이브러리로 웹 크롤러를 만들 수 있음
- requests는 특정 url로부터 html 문서를 가져오는 작업을 수행
- 나무위키와 같은 페이지는 html 문서가 javascript로 동적 로딩되는 경우가 있음
- requests 대신 셀레니움(selenium) 라이브러리를 이용해 크롬 브라우저로 동적 웹크롤링 수행
- selenium은 웹 브라우저를 자동으로 구동해주는 라이브러리
- selenium을 사용하기 위해 크롬 드라이버를 이용해 크롬 브라우저 자동으로 구동 => 크롬드라이버 필요

### [BeautifulSoup와 selenium을 이용한 웹 크롤링]
- anaconda prompt 혹은 terminal 에서 아래와 같은 패키지들을 설치
- pip install selenium
- pip install beautifulsoup4

### [크롬 브라우저 업데이트 및 크롬 드라이버 설치]
- 크롬 브라우저 설정해서 최신 버전으로 업데이트
- 크롬 드라이버 사이트에서 브라우저 버전에 맞는 드라이버 다운로드
    - https://chromedriver.chromium.org/downloads
- chromedriver.exe 파일을 노트북 파일 경로에 이동 



In [1]:
!pip install selenium

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

### BeautifulSoup의 select() VS find_all()
- html 의 특정 요소 선택
- select, select_one 의 경우 css 선택자를 이용하는 것처럼 사용 가능
- select의 경우 후손이나 자손 요소를 css처럼 선택 가능
- 예) soup.select("dl>dt>a")
- find_all, find의 경우 하나의 태그(name="table")나 하나의 클래스(class="tables")를 선택
- find 의 경우 후손이나 자손 요소를 직접 선택할 수 없어 한 번 더 변수에 담든지 루프 문을 이용해야 함
- 예) find_all(class = "ah_roll"), find(name="table")

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re #정규식 표현을 위한 모듈 

In [4]:
#윈도우용 크롬 웹드라이버 실행 경로 (windows)지정
excutable_path = "chromedriver.exe"
driver = webdriver.Chrome(executable_path = excutable_path)



#사이트의 html 구조에 기반하여 크롤링을 수행
source_url = "https://namu.wiki/RecentChanges" #크롤링할 사이트 주소를 정의
driver.get(source_url) #크롬 드라이버를 통해 url의 html 문서 가져옴


#영진씨방법
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
element = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CLASS_NAME, "app")))

# #명진씨방법
# import time
# time.sleep(10)


req = driver.page_source #전체페이지 
print(req)

<html><head><link href="/skins/senkawa/6.78e51ab2220f03bef316.css" rel="stylesheet"><link href="/skins/senkawa/3.3b036d08473e22c4f18f.css" rel="stylesheet"><script async="" src="/cdn-cgi/bm/cv/669835187/api.js"></script><style type="text/css">.resize-observer[data-v-b329ee4c]{position:absolute;top:0;left:0;z-index:-1;width:100%;height:100%;border:none;background-color:transparent;pointer-events:none;display:block;overflow:hidden;opacity:0}.resize-observer[data-v-b329ee4c] object{display:block;position:absolute;top:0;left:0;height:100%;width:100%;overflow:hidden;pointer-events:none;z-index:-1}</style><link rel="stylesheet" type="text/css" href="/skins/senkawa/11.ba5eb8d7db97b4912386.css"><script charset="utf-8" src="/skins/senkawa/11.ba5eb8d7db97b4912386.js"></script><title>최근 변경내역 - 나무위키</title><link data-n-head="1" rel="canonical" href="https://namu.wiki/RecentChanges"><link data-n-head="1" rel="search" type="application/opensearchdescription+xml" title="나무위키" href="/opensearch.xml"><

In [5]:
soup = BeautifulSoup(req, "html.parser") #BeautifulSoup 의 soup로 가공 
soup

<html><head><link href="/skins/senkawa/6.78e51ab2220f03bef316.css" rel="stylesheet"/><link href="/skins/senkawa/3.3b036d08473e22c4f18f.css" rel="stylesheet"/><script async="" src="/cdn-cgi/bm/cv/669835187/api.js"></script><style type="text/css">.resize-observer[data-v-b329ee4c]{position:absolute;top:0;left:0;z-index:-1;width:100%;height:100%;border:none;background-color:transparent;pointer-events:none;display:block;overflow:hidden;opacity:0}.resize-observer[data-v-b329ee4c] object{display:block;position:absolute;top:0;left:0;height:100%;width:100%;overflow:hidden;pointer-events:none;z-index:-1}</style><link href="/skins/senkawa/11.ba5eb8d7db97b4912386.css" rel="stylesheet" type="text/css"/><script charset="utf-8" src="/skins/senkawa/11.ba5eb8d7db97b4912386.js"></script><title>최근 변경내역 - 나무위키</title><link data-n-head="1" href="https://namu.wiki/RecentChanges" rel="canonical"/><link data-n-head="1" href="/opensearch.xml" rel="search" title="나무위키" type="application/opensearchdescription+xm

In [6]:
contents_table = soup.find(name="table") #find 함수를 이용해 태그명이 table인 것을 찾기
table_body = contents_table.find(name= "tbody") #table 안 tbody 태그인 것 찾기
table_rows = table_body.find_all(name="tr") #table tbody 안 tr태그인 것 찾기 => [ ]의 요소로 담김 
#table_body
table_rows[0]


<tr class="" data-v-88f60ae4=""><td data-v-88f60ae4=""><a data-v-88f60ae4="" href="/w/%ED%95%98%EB%8B%A4">하다</a> <a data-v-88f60ae4="" href="/history/%ED%95%98%EB%8B%A4">[역사]</a> <a data-v-88f60ae4="" href="/diff/%ED%95%98%EB%8B%A4?rev=194&amp;oldrev=193">[비교]</a> <a data-v-88f60ae4="" href="/discuss/%ED%95%98%EB%8B%A4">[토론]</a> <span data-v-88f60ae4="">(<span class="m" data-v-3ee54395="" data-v-88f60ae4="">-12</span>)</span></td> <td data-v-88f60ae4=""><div class="v-popover" data-v-0e0e98f4="" data-v-88f60ae4=""><div aria-describedby="popover_1d4rnws1cz" class="trigger" style="display: inline-block;"><a class="u" data-v-0e0e98f4="">quel</a> </div> </div> <!-- --></td> <td data-v-88f60ae4=""><time data-v-88f60ae4="" datetime="2022-01-13T06:23:20.000Z">2022-01-13 15:23:20</time></td></tr>

In [7]:
len(table_rows)

112

### [페이지 링크주소 리스트 가져오기]

In [12]:
#특성 속성 값을 추출
page_url_base = "https://namu.wiki" #베이스 url 정의
page_urls = [] # href 속성값을 담기 위한 빈 리스트  

for i in range(0, len(table_rows)): #table_rows의 길이만큼 반복
    first_td = table_rows[i].find_all('td')[0] #td가 3개 있는데 0번째에 원하는 href 가 있음 
    td_url = first_td.find_all('a')
    if len(td_url) > 0:
        page_url = page_url_base + td_url[0].get('href') #나무위키주소+get() 태그가 가지고 있는 속성 추출 
        if "png" not in page_url:
            page_urls.append(page_url)
          
        
    page_urls = list(set(page_urls)) #중복 url 제거
    for page in page_urls[:3]:
        print(page)



https://namu.wiki/w/%ED%95%98%EB%8B%A4
https://namu.wiki/w/%EC%B9%B8%EB%AA%AC%20%ED%95%B4%ED%98%91
https://namu.wiki/w/%ED%95%98%EB%8B%A4
https://namu.wiki/w/%EC%B9%B8%EB%AA%AC%20%ED%95%B4%ED%98%91
https://namu.wiki/w/%ED%95%98%EB%8B%A4
https://namu.wiki/w/%EC%97%AD%EC%B2%B4%EC%9B%90
https://namu.wiki/w/%EC%B9%B8%EB%AA%AC%20%ED%95%B4%ED%98%91
https://namu.wiki/w/%ED%95%98%EB%8B%A4
https://namu.wiki/w/%EC%97%AD%EC%B2%B4%EC%9B%90
https://namu.wiki/w/%EC%B9%B8%EB%AA%AC%20%ED%95%B4%ED%98%91
https://namu.wiki/w/%ED%95%98%EB%8B%A4
https://namu.wiki/w/%EC%97%AD%EC%B2%B4%EC%9B%90
https://namu.wiki/w/%ED%95%98%EB%8B%A4
https://namu.wiki/w/%E9%99%BD%E3%81%A0%E3%81%BE%E3%82%8A%E3%83%AD%E3%83%BC%E3%83%89%E3%83%8A%E3%82%A4%E3%83%88
https://namu.wiki/w/%EC%B9%B8%EB%AA%AC%20%ED%95%B4%ED%98%91
https://namu.wiki/w/%ED%95%98%EB%8B%A4
https://namu.wiki/w/%E9%99%BD%E3%81%A0%E3%81%BE%E3%82%8A%E3%83%AD%E3%83%BC%E3%83%89%E3%83%8A%E3%82%A4%E3%83%88
https://namu.wiki/w/%EC%B9%B8%EB%AA%AC%20%ED%95%B4%ED%98%91
h